In [2]:
import pandas as pd
import h2o
from sklearn.preprocessing import LabelEncoder 
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,55 secs
H2O_cluster_timezone:,Asia/Yekaterinburg
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,2 months and 7 days
H2O_cluster_name:,H2O_from_python_kirillkuznecov_5jx4cn
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.770 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
df = pd.read_csv("./data/mushrooms.csv")

# Загрузка данных в h2o фрейм
# df = h2o.import_file("./data/mushrooms.csv", header =1)

In [4]:
labelEncoder = LabelEncoder()
for column in df.columns:
    df[column] = labelEncoder.fit_transform(df[column])
df = h2o.H2OFrame(df)
df = df.asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Подобно функциям в sklearn, мы можем создать разбиение на train и test, чтобы можно было проверить производительность модели на неизвестном наборе данных. 

In [5]:
# Разделяем данные на Train/Test/Validation. Train размером 70%, test и validation по 15% 
train_df, test_df = df.split_frame(ratios=[.7])

Затем нам нужно получить имена столбцов, чтобы передать их функции. 

Для AutoML необходимо указать несколько обязательных параметров: x, y и training_frame. Здесь вы также можете настроить значения max_runtime_sec и max_models. 

Max_runtime_sec — обязательный параметр, а max_model — необязательный. 
По умолчанию все непереданные параметры принимают значение NULL. 


Параметр x — это вектор признаков из training_frame. Если вы не хотите использовать все признаки из переданного кадра, вы можете установить его, передав его в x.

На этих данных будем использовать все признаки (кроме таргета) и установим max_runtime_sec на 10 минут (по умолчанию некоторые из моделей могут занять много времени). Запустим AutoML:

In [6]:
y = "class"
train_columns = train_df.columns
train_columns.remove(y)

aml_model = H2OAutoML(max_runtime_secs=500, seed = 1)
aml_model.train(x = train_columns, y = y, training_frame = train_df)

AutoML progress: |
17:38:59.259: AutoML: XGBoost is not available; skipping it.
17:38:59.359: _train param, Dropping bad and constant columns: [veil-type]

████
17:39:01.147: _train param, Dropping bad and constant columns: [veil-type]

██
17:39:28.167: _train param, Dropping unused columns: [veil-type]


17:39:29.833: _train param, Dropping bad and constant columns: [veil-type]

█
17:39:31.340: _train param, Dropping bad and constant columns: [veil-type]

██
17:39:47.222: _train param, Dropping bad and constant columns: [veil-type]

█
17:40:00.830: _train param, Dropping bad and constant columns: [veil-type]

██
17:40:16.331: _train param, Dropping unused columns: [veil-type]
17:40:17.70: _train param, Dropping unused columns: [veil-type]


17:40:18.516: _train param, Dropping bad and constant columns: [veil-type]

█
17:40:20.753: _train param, Dropping bad and constant columns: [veil-type]

███
17:40:32.746: _train param, Dropping bad and constant columns: [veil-type]


17:40:34.457:

key,value
Stacking strategy,cross_validation
Number of base models (used / total),2/3
# GBM base models (used / total),1/1
# DRF base models (used / total),1/1
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None
Custom metalearner hyperparameters,None


Мы задали значение параметра Max_runtime_sec ~10 минут, но также есть параметр, отвечающий не за время, а за максимальное количество исследованных моделей. Помимо этого, есть возможности дополнительной настройки процесса работы AutoML и множество дополнительных параметров, которые вы можете задать. 

После запуска и обучения H2OAutoML вы можете посмотреть, какие модели в нашем случае сработали лучше всего, и выбрать их для дальнейшего исследования.

In [7]:
leaderboard = aml_model.leaderboard
leaderboard.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_2_AutoML_1_20240227_173859,1,0.000802009,1,0,0.00092003,8.46456e-07
GBM_grid_1_AutoML_1_20240227_173859_model_7,1,4.92055e-18,1,0,1.94537e-16,3.78446e-32
GBM_3_AutoML_1_20240227_173859,1,9.00228e-17,1,0,5.0177e-15,2.51773e-29
StackedEnsemble_AllModels_2_AutoML_1_20240227_173859,1,0.000677473,1,0,0.000679485,4.617e-07
GBM_grid_1_AutoML_1_20240227_173859_model_10,1,2.90855e-16,1,0,1.57747e-14,2.4884e-28
GBM_1_AutoML_1_20240227_173859,1,2.77159e-16,1,0,2.07018e-14,4.28567e-28
GBM_grid_1_AutoML_1_20240227_173859_model_16,1,5.7641e-06,1,0,6.65035e-05,4.42271e-09
StackedEnsemble_AllModels_1_AutoML_1_20240227_173859,1,0.000677441,1,0,0.000679447,4.61648e-07
GBM_grid_1_AutoML_1_20240227_173859_model_1,1,9.60121e-15,1,0,5.36806e-13,2.88161e-25
StackedEnsemble_BestOfFamily_1_AutoML_1_20240227_173859,1,0.000847286,1,0,0.000847465,7.18197e-07


Давайте проверим результат предсказания на тестовых данных и посмотрим, не переобучилась ли модель:

In [10]:
prediction = aml_model.predict(test_df)
prediction

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.999488,0.000512104
0,0.999488,0.000512104
1,0.000582984,0.999417
0,0.999488,0.000512104
0,0.999488,0.000512104
0,0.999488,0.000512104
1,0.000582984,0.999417
0,0.999488,0.000512104
0,0.99841,0.00158954
0,0.999488,0.000512104


In [ ]:
h2o.shutdown()

Для наших данных H2O AutoML показал отличные результаты на тестовой выборке. Это значит, что модель не переобучилась (в случае переобучения на test мы бы имели низкую оценку, а на train — высокую).

Получив лучшую (из исследуемых) модель, можно выполнить несколько шагов, которые могут повысить производительность модели (на следующем этапе после MVP):

Подготовка признаков (feature engineering).
Добавление новых признаков с учётом имеющихся зависимостей и выявление новых (feature extraction).
Отбор наиболее важных признаков для модели (feature selection). 
Многие из лучших моделей в H2O AutoML используют ансамблирование. Эти модели можно дополнительно настроить.

✔️ Несмотря на то, что AutoML не принесёт вам призовых мест в соревнованиях по машинному обучению на Kaggle или лучших результатов для рабочих задач, его определённо стоит рассмотреть в качестве дополнения к вашим простым моделям и бейзлайну.

✔️  AutoML может обрабатывать множество различных типов наборов данных, включая бинарную классификацию (как мы посмотрели выше), многоклассовую классификацию, а также задачи регрессии.

✔️  AutoML — отличный инструмент, помогающий в работе (а не заменяющий её). Но когда дело доходит до feature engineering, такой автоматизированный инструмент не может превзойти творческое мышление человека. 